Суть идеи проста. При при помощи модуля diff из библиотеки Sympy автоматизируем процесс нахождения формул частных производных!!! По полученным формулам вычисляем поправки и осущаствяем градиентый спуск.
- Имеются функции активации: Sigmoid (Сигмода) и Tahn(Гиперболический тангенс). Легко добавить любые другие.
- 1 функция потерь (квадратичная). Легко добавить другие.
- Имеется небольшая оптимизация Learning Rate.  

ГЛАВНЫЙ МИНУС ! Не реализован алгоритм обратного распространения ошибки !!! (
     
Sympy поддерживает numpy-массивы, но проиводительность низкая из-за однопоточных рассчетов. 

В данном виде проект совершенно не доработан. ПРОШУ ПРОЩЕНИЯ !

In [40]:
import math

import numpy as np
from sklearn import datasets
import matplotlib.pyplot as plt

%matplotlib inline


from sympy import diff, lambdify, symbols
import re
from copy import deepcopy

In [41]:
# формируем датасэт
X, Y = datasets.make_regression(n_samples=10000, n_features = 60, n_informative = 2, n_targets = 1, 
                                              noise = 25, coef = False, random_state = 2) 

In [19]:
def standart(data):  # Стандартизация 
    means = np.mean(X, axis=0)
    stds = np.std(X, axis=0)

    for i in range(data.shape[0]):
        for j in range(data.shape[1]):
            data[i][j] = (data[i][j] - means[j])/stds[j]
    return data

In [20]:
def normal(data): # Нормализация 
    for i in range (data.shape[0]):
        data_max_min = np.max(data[i]) - np.min(data[i])  
        data_min = np.min(data[i]) 
        for j in range (data.shape[1]):
            data[i][j] = (data[i][j] - data_min)/data_max_min
    return data

In [21]:
# Функции создания слоев сети


# Создание входного слоя, функция принимает количество входов (фичей) и количество нейронов в слое,
# функцию активации. 
# ДОСТУПНЫ Sigmoid, Tahn и None (без функции активации)
# Формирутся и возвращается формула(-ы) ввиде строки (str). 

def add_input_layer(inputs, neurons, activation):
    num_inputs = []
    num_w = []
    neuron = []

    for _ in range( inputs):

        num_inputs.append('x'+str(_+1))
    
        num_w.append('w'+str(_+1))



    
    for _ in range(neurons):
        neuron.append('b'+str(_+1))
    #print(neuron) ############


    layer = []
    for n in neuron:
        st =''
        for perem in range(len(num_inputs)):
            st = st + num_inputs[perem]+' * '+ num_w[perem]+' + '
        
        if activation == 'None':
            layer.append  (st + n ) # линейная
        elif activation == "Sigmoid":
            layer.append  ('1/1+'+str(2.718)+'**-('+st + n+')'  ) # добавление СИГМОИДНОЙ функции активации
            
        elif activation=="Tahn":
            layer.append('('+str(2.718)+'**(2*('+st + n+'))-1)/('+str(2.718)+'**(2*('+st + n+'))+1)' )
    
    
    #print(st)

        
    n = 1

    while n< len(layer):
    
        for i in range(inputs): 
            pat = 'w'+ str(i+1)+' '
            su =  'w' + str(n*inputs+i+1)+' '
            layer[n] = re.sub(pat , su     ,layer[n])

  
        n+=1

    return layer



# Создание дополнительных слоев. В качестве агрументов принимает массив (list), созданый функцией "add_input_layer", 
# количестово нейронов и activation . Возвращает массив (list) формул нейронной сети.


def add_layer(layer, neurons,activation):
    num_inputs = []
    num_w = []
    neuron = []

    for _ in range(len(layer)):

        num_inputs.append('c'+str(_+1))
    
        num_w.append('w'+str(_+1))


    
    b_kol_1 = re.findall("b\d+", layer[-1])
    b_max = int(b_kol_1[-1][1:])
    #print("Кол-во нейронов на предыдущем слое", b_max) ########



    
    for _ in range(neurons):
        neuron.append('b'+str(b_max+_+1))



    layer2 = []
    for n in neuron:
        st =''
        for perem in range(len(num_inputs)):
            st = st + num_inputs[perem]+' * '+ num_w[perem]+' + '

            
        if activation=="None":
            layer2.append  ( '('+st + n+')'  )
        elif activation=="Sigmoid":
            layer2.append  ('1/1+'+str(2.718)+'**-('+st + n+')'  )
        
        elif activation=="Tahn":
            layer2.append('('+str(2.718)+'**(2*('+st + n+'))-1)/('+str(2.718)+'**(2*('+st + n+'))+1)' )
    
    
    
    
    n = 0
    m = 1

    w_kol_1 = re.findall("w\d+", layer[-1])
    w_max = int(w_kol_1[-1][1:])
    #print("Кол-во весов на предыдущем слое", w_max) ######
    
    while n< len(layer2):
    
        for i in range(len(layer)):

        
            pat = 'w'+ str(i+1)+' '
            su =  'w' + str(w_max+m)+' '

            layer2[n] = re.sub(pat , su     ,layer2[n])
            m+=1

        #print (n, "!!!!!", (n)*len(layer)+i+1 ,"!!!!!", pat, layer2[n]) ####

       
    
        n+=1

    #print(layer2)
    
    
    
    n = 0
    for _ in range(len(layer2)):
        while n< len(layer):
            pat = 'c'+ str(n+1)+' '
            su =  '(' +layer[n]+')' +' '
            layer2[_] = re.sub(pat , su ,layer2[_])
            n+=1


        n=0
    
    #print(layer2)
    
    return layer2


        


In [42]:
# ПРИМЕР СОЗДАНИЯ НЕЙРОННОЙ СЕТИ

# net = add_input_layer (5, 3, activation="Sigmoid") 5 входов, 3 нейрона, Активация "Сигмоида".
# net = add_layer (net, 3, activation="Tahn") Подключение скрытого(или выходного) слоя к предыдущему. Передаем количетво нейронов в слое.
# net = add_layer (net, 1, activation="None") В данном случае создается выходной слой с 1 нейроном.

# ЕСЛИ В СЕТИ ТОЛЬКО 1 СЛОЙ - неодходима только функция  add_input_layer 

In [43]:
# Создание слоя. Количестово входов = количесто фичей. Создадим 1 нейрон т.е. 1 формулу. БЕЗ функции активации
net = add_input_layer (X.shape[1], 1, activation="None")


In [44]:
# Так как созданый объект представляет из себя список (list). В качетве формулы сети забираем его 1-й элемент.
net2 = net[0] 
net

['x1 * w1 + x2 * w2 + x3 * w3 + x4 * w4 + x5 * w5 + x6 * w6 + x7 * w7 + x8 * w8 + x9 * w9 + x10 * w10 + x11 * w11 + x12 * w12 + x13 * w13 + x14 * w14 + x15 * w15 + x16 * w16 + x17 * w17 + x18 * w18 + x19 * w19 + x20 * w20 + x21 * w21 + x22 * w22 + x23 * w23 + x24 * w24 + x25 * w25 + x26 * w26 + x27 * w27 + x28 * w28 + x29 * w29 + x30 * w30 + x31 * w31 + x32 * w32 + x33 * w33 + x34 * w34 + x35 * w35 + x36 * w36 + x37 * w37 + x38 * w38 + x39 * w39 + x40 * w40 + x41 * w41 + x42 * w42 + x43 * w43 + x44 * w44 + x45 * w45 + x46 * w46 + x47 * w47 + x48 * w48 + x49 * w49 + x50 * w50 + x51 * w51 + x52 * w52 + x53 * w53 + x54 * w54 + x55 * w55 + x56 * w56 + x57 * w57 + x58 * w58 + x59 * w59 + x60 * w60 + b1']

In [45]:
# формируем список всех переменных имеющихся в функции сети и сортируем. Все X должны быть в конце.
li_perem = sorted(list(set(re.findall("[w,x,b]\d+"    ,net2))))
print(li_perem)

['b1', 'w1', 'w10', 'w11', 'w12', 'w13', 'w14', 'w15', 'w16', 'w17', 'w18', 'w19', 'w2', 'w20', 'w21', 'w22', 'w23', 'w24', 'w25', 'w26', 'w27', 'w28', 'w29', 'w3', 'w30', 'w31', 'w32', 'w33', 'w34', 'w35', 'w36', 'w37', 'w38', 'w39', 'w4', 'w40', 'w41', 'w42', 'w43', 'w44', 'w45', 'w46', 'w47', 'w48', 'w49', 'w5', 'w50', 'w51', 'w52', 'w53', 'w54', 'w55', 'w56', 'w57', 'w58', 'w59', 'w6', 'w60', 'w7', 'w8', 'w9', 'x1', 'x10', 'x11', 'x12', 'x13', 'x14', 'x15', 'x16', 'x17', 'x18', 'x19', 'x2', 'x20', 'x21', 'x22', 'x23', 'x24', 'x25', 'x26', 'x27', 'x28', 'x29', 'x3', 'x30', 'x31', 'x32', 'x33', 'x34', 'x35', 'x36', 'x37', 'x38', 'x39', 'x4', 'x40', 'x41', 'x42', 'x43', 'x44', 'x45', 'x46', 'x47', 'x48', 'x49', 'x5', 'x50', 'x51', 'x52', 'x53', 'x54', 'x55', 'x56', 'x57', 'x58', 'x59', 'x6', 'x60', 'x7', 'x8', 'x9']


In [46]:
# создание символов для SYMPY на основе списка. Получается список из SYMPY классов

li =[]
for p in li_perem: 
    symbols(p)
    li.append(symbols(p))

# для вычисления ошибки добадляем в список название переменной правильного ответа (у). Он должен быть последним.
y = symbols('y1')
li.append(symbols('y1'))
print(li)

[b1, w1, w10, w11, w12, w13, w14, w15, w16, w17, w18, w19, w2, w20, w21, w22, w23, w24, w25, w26, w27, w28, w29, w3, w30, w31, w32, w33, w34, w35, w36, w37, w38, w39, w4, w40, w41, w42, w43, w44, w45, w46, w47, w48, w49, w5, w50, w51, w52, w53, w54, w55, w56, w57, w58, w59, w6, w60, w7, w8, w9, x1, x10, x11, x12, x13, x14, x15, x16, x17, x18, x19, x2, x20, x21, x22, x23, x24, x25, x26, x27, x28, x29, x3, x30, x31, x32, x33, x34, x35, x36, x37, x38, x39, x4, x40, x41, x42, x43, x44, x45, x46, x47, x48, x49, x5, x50, x51, x52, x53, x54, x55, x56, x57, x58, x59, x6, x60, x7, x8, x9, y1]


In [47]:
%%time

# САМОЕ ВАЖНОЕ. Производные

# Используем регуляторные выражения, чтобы сформировать список названий переменных по которым будем брать производные

li_for_diff = sorted(list(set(re.findall("[w,b]\d+"    ,net2)))) # выбирам названия начиющиеся с "w" - вес, и "b" - смещение


#В цикле передам функции sympy.diff формулу ошибки и название переменной по которой берем производную.
# Получаем список производых
li_dif = []
for i in li_for_diff:
    proizvod = diff(  "("+net2+"-y1)**2" , i)
    li_dif.append(proizvod)





Wall time: 313 ms


In [48]:
# инициализирум веса и смещение случайным числами

zna = []
for _ in range(len(li_perem)-X.shape[1]):
    zna.append(np.random.normal())


    
li_x_y = sorted(list(set(re.findall("[x,y]\d+"   , str(li) ))))


# добавляем к полученному массиву наш датасэт
for i in li_x_y:
    if i[0] == "x":
        
        a = int(i[1:])

        zna.append(X.T[a-1]) # добавляем иксы 
    elif i[0] == "y":
        b = int(i[1:])

        zna.append(Y.T[b-1]) # добавляем правильные ответы
        



In [49]:
%%time

# просто проверка работоспособности и производительности

# вычислим лосс
loss_funk = lambdify([li], "("+net2+"-y1)**2", 'numpy')
loss = np.sum(loss_funk(zna))/Y.size
print(loss)

# вычислим поправку по первому коэффициенту
proiz = lambdify([li], li_dif[0] , 'numpy')

grad = np.sum(proiz(zna))/Y.size
print(grad)


911.5215511161406
-57.88334656497993
Wall time: 61.1 ms


In [50]:
%%time



lr = 0.01

lr_primary = deepcopy(lr)



loss_history = np.array([])



while True: # ну пусть так 
    
    
    loss_funk = lambdify([li], "("+net2+"-y1)**2", 'numpy') # вычислим лосс
    loss = np.sum(loss_funk(zna))/Y.size
    print(loss)
    
    
    
    loss_history = np.append(loss_history, loss) # небольшая оптимизация Learning Rate
    if len(loss_history)>=2:
        if loss_history[-1]> loss_history[-2]:
            lr=lr/10
            print("LEARNING RATE = ",  lr) 
            
        elif loss_history[-2]*0.90<loss_history[-1]: # здесь я выбрал 90%
            lr=lr+lr_primary
            print("LEARNING RATE = ", lr)
    
    
    
    s = 0
    for i in li_dif: # перем с списка формулы производных и считаем поправку (градиент) для каждого
        
        proiz = lambdify([li], i , 'numpy') # создается функция. Передаем список аргументов, формулу производной. Формат Numpy

        grad = np.sum(proiz(zna))/Y.size  # усредняем поправку
    
        zna[s] = zna[s] - lr*grad # делаем шаг
        s+=1
    
    
    
    
       
    


911.5215511161406
875.0229550276858
LEARNING RATE =  0.02
805.6919417527066
LEARNING RATE =  0.03
710.977213956669
627.4427922194593
553.7635644566934
488.772101111363
431.43981820834875
380.86039928485417
336.23520431254775
296.8604265697187
262.1157872832084
231.45458322448343
204.3949247370625
180.51202126686354
159.43138869066055
140.82286787854048
124.39535723679909
109.89217368008931
97.08696677081346
85.78011981098203
75.79557962748255
66.97806378805664
59.19060013837293
52.312358962400424
46.23674282845829
40.869703370581
36.12825793840057
31.93918228941349


KeyboardInterrupt: 